# Classification of Bottle Openers, Can Openers and Corc Screws by Means of AI Methods

This assignment aims to provide multiple methods for classifying bottle openers, can openers and corc screws from images. The results of the different methods are to be analyzed and compared.

## 1. Data Aquisition and Augmentation

To train the methods for classification from images, first some training data has to be acquired. 

## 2. Feature Extraction

The following features are to be extracted from the original image dataset (without the augmented images)
* The outer contour's aspect ratio
* Number of corners detected via Harris Corner Detection
* Number of corners detected via Shi-Tomasi Corner Detection
* The outer contour's perimeter-area ratio

After the features are extracted they are saved in a CSV-File.

### Required Imports

In [ ]:
import os
import pandas
import cv2
import numpy as np

import modules.feature_extraction as fe